# Computation Theory

- - -

In [2]:
#imports
import numpy as np

## Problem Markdown Structure

**The following will detail how I will structure and organise my approach to each of the problems.**

### 1. Title: Problem X - Short Topic Name

Example: *'Problem 2 – Finding Cuberoots'*

### 2. Problem Description

- Will include a copy of the full question I am being asked to solve.

- Following with my understanding of the task I am given.

### 3. Theory / Background

Explanation of the relevant concepts.

**For Example:**

- What is **SHA-256**?
- How **SHA-256** was made.
- When was **SHA-256** invented?

### 4. Approach

Explain **how** I solved the question and **why** I solved it the way I did.

**For Example**

- Represent the message as a sequence of bytes and compute its original bit length.

- Append the 64-bit big-endian length field to complete the padded message

- Use integer operations to represent binary patterns.

### 5. Implementation (Code Cell)

This is where my actual ``Python`` code implementation exists. In many or some instances I may have multiple implementations or experimentation cells. 

### 6. Python Docstring (Code Cell)

I will be using a **NumPy docstring documentation style**, which is widely used in scientific Python and considered on of the cleaned and most professional.

*It looks as follows:*

```python
"""
Brief one-line summary of what the function does.

Extended description explaining the purpose of the function,
how it works, and any important details about the logic or
constraints (e.g., must operate on 32-bit words, follows
SHA-256 specification rules, etc.).

Parameters
----------
param1 : int or np.uint32
    Explanation of the first parameter and its expected type.
param2 : int or np.uint32
    Explanation of the second parameter.

Returns
-------
np.uint32
    Description of the returned value and what it represents.

Notes
-----
- behaviour defined by the Secure Hash Standard (FIPS 180-4)
- bitwise rotations wrap around within 32 bits
- numpy ensures correct unsigned 32-bit behaviour

Examples
--------
>>> function_name(0b1010, 0b1100)
0b0110
"""
```

### 7. Test Cases / Outputs

Often ``Python`` **test cases** that test my implementations and give an output. It could show a printed output, diagrams or even graphs. The test cases should be always placed **under** the problem's implementation, **with the exception of problem 1**. This is because problem 1 required multiple functions written so I felt it was more readable to place test cases under each function. 

### 8. Discussion / Interpretation    

Explain what the actual results mean and my interpreation of my approach.

**For Example:**
- The padded block structure matches the Secure Hash Standard, ensuring compatibility with real SHA-256 implementations.

- The final two blocks appear only when the original message length forces the padding to overflow the first block.

- Bitwise padding rules enforce canonical representation of all possible inputs, preventing ambiguity and collisions.

### 9. References (URLs + 1 sentence explanation)

**References** and **sources** that I used over the above question. Every reference requires a short sentence about how I used it.

**For Example:**
- "Python docs: https://docs.python.org - used for generator syntax."

- - - 


 

## Symbol Notation
---
| Symbol | Name | Operation Description | Example `(x = 0b1100, y = 0b1010)`| 
|:-:|:-:|:-:|:-----------------:|
| & | Bitwise AND | Compares each bit of two numbers. Bit is *1* only if both bits are *1*. | `x & y = 0b1000` | 
| ` | Bitwise OR |Compares each bit of two numbers. Bit is *1* if either bit is *1*. | ``x ` y = 0b1110`` | 
| ^ | Bitwise XOR | Bit is 1 if the two bits are *different*. | `x ^ y = 0b0110` | 
| ~ | Bitwise NOT | Flips every bit *(1 > 0, 0 > 1)*. Works as *-(x+1)* in Python due to two’s complement. | `~x = -0b1101` |
| << | Left Shift | Shifts bits to the left by *n* places. Fills in zeros on the right. | `x << 2 = 0b110000` | 
| >> | Right Shift | Shift bits to the right by *n* places. Fills in the zeros on the left. | `x >> 2 = 0b0011` |  

# Problem 1: Binary Words and Operations

---

## Problem Description

Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the [Secure Hash Standard](https://csrc.nist.gov/pubs/fips/180-2/final) (see page 10).

### Required Functions:

| Function | Standard Notation | Description |
|:---------|:------------------|:------------|
| `Parity(x, y, z)` | - | XOR-based parity function |
| `Ch(x, y, z)` | - | Choose function |
| `Maj(x, y, z)` | - | Majority function |
| `Sigma0(x)` | Σ₀²⁵⁶(x) | Upper-case Sigma 0 (rotations: 2, 13, 22) |
| `Sigma1(x)` | Σ₁²⁵⁶(x) | Upper-case Sigma 1 (rotations: 6, 11, 25) |
| `sigma0(x)` | σ₀²⁵⁶(x) | Lower-case sigma 0 (rotations: 7, 18; shift: 3) |
| `sigma1(x)` | σ₁²⁵⁶(x) | Lower-case sigma 1 (rotations: 17, 19; shift: 10) |

Document each function with a **clear docstring**, explain its **purpose and behaviour** in *Markdown*, and test it with appropriate examples to verify correctness.

## My Understanding

For this problem, I need to re-create some of the core building blocks used inside the **SHA-256 hashing algorithm**. The **SHA-256** algorithm is a **cryptographic hash function** that turns a given input into a series of hexadecimal values **(fixed 256-bit hash)**. Hashes, unlike encryptions, are long and **can not be reversed**. One input will always output the same hash code making hashing extremely useful for something like storing and validating passwords. These blocks are small functions that work on 32-bit binary words, and they mostly use bitwise logic like ``XOR``, ``AND``, ``OR``, **rotations**, and **shifts**.

*Each function has a specific job in the hash process:*

- ``Parity``, ``Ch``, and ``Maj`` take **three 32-bit** values and **combine** them using logic rules **(XOR, choose-between, or majority vote)**.

- ``Sigma0`` and ``Sigma1`` (uppercase) rotate the bits of a number by certain fixed amounts and ``XOR`` the results together.

- ``sigma0`` and ``sigma1`` (lowercase) also use rotations, but each one includes a right-shift as well.

All the operations have to be done using **32-bit integers**, so I need to use **NumPy** to make sure the values wrap around properly. Once I write each function, I should document what it does and test it with example values to confirm that the outputs match what the standard expects.

---

## Theory / Background 

**SHA-256** is a **cryptographic hash function** from the **Secure Hash Standard**. Its job is to take any input and turn it into a **fixed 256-bit hash value**. This value is **one-way**, meaning you can’t reverse it to get the original message. It’s also **deterministic**, so the same input always gives the same output, and it’s designed to **avoid collisions**, where two different inputs produce the same hash.

**SHA-256** was developed by the **National Institute of Standards and Technology (NIST)** and released in **2001** as part of the **Secure Hash Standard (FIPS PUB 180-2)**. It was introduced to replace older hash functions like **SHA-1**(which is now deprecated) and to provide a stronger, modern hashing method for security, digital signatures, and data integrity.

The algorithm works by splitting the message into **512-bit blocks** and running them through a series of bitwise operations like ``XOR``, ``AND``, **rotations**, and **shifts**. Functions such as ``Ch``, ``Maj``, and the ``Sigma`` functions help mix the bits so that even a 1-bit change in the input causes a completely different output. This is called the **avalanche effect**.

**SHA-256** is widely used for **checking data integrity**, **digital signatures**, and **general security tasks**. 

- - -

## Aproach

### 1. Parity Function

For the **Parity function**, I followed the definition from the **SHA standard**, which states that parity is computed using a bitwise **XOR** across the three input values. I first converted ``x``, ``y``, and ``z`` to ``np.uint32`` to ensure they behave as **32-bit words**, since **SHA-256** relies on strict 32-bit operations. After that, I applied **XOR** to all three values, because **XOR** naturally produces the correct parity: **each bit in the output becomes 1 if an odd number of the input bits are 1**. This matches the behaviour needed for SHA-style logical functions and keeps the implementation simple and accurate.

I solved it this way because **XOR naturally implements parity: it sets a bit to 1 only when an odd number of the input bits are 1**, which is exactly what the **SHA** standard requires. Converting the inputs to ``np.uint32`` ensures the function behaves like a true 32-bit operation, matching how **SHA-256** handles all of its internal values. 

### 2. Choose Function

For the **Choose function**, I followed the formula defined in the **SHA-256** standard, where each output bit is selected from either ``y`` or ``z`` depending on the corresponding bit in ``x``. If a bit in ``x`` is **1**, the bit from ``y`` **is chosen**, and if it is **0**, the bit from ``z`` **is chosen**. To match the behaviour of the algorithm, I first **converted all inputs to ``np.uint32`` so that the operation stays within 32-bit boundaries**. After that, I implemented the expression ``(x & y) ^ (~x & z)``, which is the exact **logical form used inside the SHA-256 compression function**. This ensures that each bit is selected correctly based on ``x`` and follows the rules laid out in the official specification.

I solved it this way because this expression is the direct translation of how the **Choose function** is defined in the **SHA standard**. The combination of bitwise **AND**, **NOT**, and **XOR** cleanly selects between ``y`` and ``z`` using the bits of ``x``, and using ``np.uint32`` **keeps everything consistent** with **SHA-256’s** strict 32-bit word operations.

### 3. Majority Function

For the **Majority function** (`Maj`), I followed the rule from the **SHA-256** standard, which says that **each bit in the output should be the value that appears in at least two of the three inputs**. To make sure the function behaves like **SHA-256**, I first converted ``x``, ``y``, and ``z`` to ``np.uint32`` so they act as proper **32-bit words**.

I solved it this way because the expression ``(x & y) ^ (x & z) ^ (y & z)`` is the exact formula the standard uses. It **checks all three inputs bit-by-bit and picks the majority value**. Using ``np.uint32`` keeps the operations inside **32 bits** and makes sure the behaviour matches what actually happens inside the **SHA-256** algorithm.

### 4. Sigma0 Function

For the ``Sigma0`` function, I used the formula from the **SHA-256** standard, which says that ``Sigma0`` is created by **rotating the input word three times: by 2 bits, 13 bits, and 22 bits**, and then **XORing** all of those results together. Before doing the rotations, I converted the input to ``np.uint32`` so it always behaves like a **32-bit word**, just like **SHA-256** requires.

I solved it this way because **SHA-256 depends on bitwise rotations to mix the bits of the message**. Using the **three rotations** and **XOR** gives the spreading effect the algorithm needs, and converting the value to ``np.uint32`` makes sure the rotations wrap around correctly and stay within 32 bits.

### 5. Sigma1 Function

For the ``Sigma1`` function, I followed the formula from the **SHA-256** standard, which defines ``Sigma1`` by **rotating the input word three times**: by **6 bits, 11 bits, and 25 bits**, and then **XORing** the results together. Just like the other **SHA functions**, I first converted the input to ``np.uint32`` so it behaves as a proper **32-bit word** and the rotations wrap around correctly.

I solved it this way because ``Sigma1`` is one of the main mixing functions in **SHA-256**, and these specific rotations help spread the bits of the word in a controlled way. The combination of the three rotations and **XOR** creates strong bit mixing, which is important for the security of the hash. Converting the input to ``np.uint32`` ensures the function matches the exact **32-bit** behaviour defined in the **SHA-256** specification.

### 6. sigma0 Function

For the **small** ``sigma0`` function, I followed the formula from the **SHA-256** standard, which uses **two right-rotations (by 7 bits and 18 bits) and a right-shift by 3 bits**. After converting the input to ``np.uint32``, I applied these operations and combined the results using **XOR**, just like the specification requires.

I solved it this way because **small sigma** functions are used in the message schedule of **SHA-256** to help expand and mix the message words. The mix of rotations and a shift creates the variation the algorithm needs, and converting to ``np.uint32`` keeps everything inside proper 32-bit behaviour so the function matches the real **SHA-256** process.

### 7. sigma1 Function

For the **small** ``sigma1`` function, I used the formula from the **SHA-256** standard, which applies **two right-rotations to the input word (by 17 bits and 19 bits) and then a right-shift by 10 bits**. After converting the input to ``np.uint32``, I **combined all three results using XOR**, exactly as the specification describes.

I solved it this way because the **small sigma** functions are **used in the message schedule part of SHA-256**, where the algorithm expands and mixes the message words. The **rotations and the shift help create variation in the bits**, and using ``np.uint32`` keeps everything inside proper 32-bit boundaries, which is required for the correct behaviour of the algorithm.


## Parity Function Implementation

- - - 

In [2]:
def Parity(x,y,z):
    """
    Computes the parity (bitwise XOR) of three 32-bit unsigned integers.
    
    Used as a logical function in SHA-1 algorithm. The parity is 1 if an 
    odd number of the three bits are 1, and 0 if an even number are 1.

    Parameters
    ----------
    x : int or np.uint32
        First integer input.
    y : int or np.uint32
        Second integer input.
    z : int or np.uint32
        Third integer input.

    Returns
    -------
    np.uint32
        The bitwise XOR of x, y, and z as a 32-bit unsigned integer.

    Examples
    --------
    >>> Parity(0b1010, 0b1100, 0b0011)
    np.uint32(5)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32(x ^ y ^ z)

### Parity Function Test Cases

- - -

In [5]:
test_cases = [
    (0, 0, 0),  # Expected: 0
    (1, 0, 0),  # Expected: 1
    (1, 1, 0),  # Expected: 0
    (1, 1, 1),  # Expected: 1
    (0b1010, 0b1100, 0b0110),  # Expected 0
    (0xFFFFFFFF, 0x00000000, 0xFFFFFFFF),  # Expected: 0 
]

for x, y, z in test_cases:
    result = Parity(x,y,z)
    print(f"Parity({x}, {y}, {z}) = {result:04b}")

Parity(0, 0, 0) = 0000
Parity(1, 0, 0) = 0001
Parity(1, 1, 0) = 0000
Parity(1, 1, 1) = 0001
Parity(10, 12, 6) = 0000
Parity(4294967295, 0, 4294967295) = 0000


## Choose Function Implementation

---

In [ ]:
def Choose(x,y,z):
    """
    Chooses bits from y or z based on the bits of x.
    
    If a bit in x is 1, the corresponding bit from y is chosen.
    If a bit in x is 0, the corresponding bit from z is chosen.
    This is a core logical function used in SHA-256 compression.

    Parameters
    ----------
    x : int or np.uint32
        Control input that determines which bits to choose.
    y : int or np.uint32
        First source input (chosen when x bit is 1).
    z : int or np.uint32
        Second source input (chosen when x bit is 0).

    Returns
    -------
    np.uint32
        Result of the expression (x & y) ^ (~x & z) as a 32-bit unsigned integer.

    Examples
    --------
    >>> Choose(0b1111, 0b1010, 0b0101)
    np.uint32(10)  # Binary: 0b1010 (all bits from y since x is all 1s)
    
    >>> Choose(0b0000, 0b1010, 0b0101)
    np.uint32(5)  # Binary: 0b0101 (all bits from z since x is all 0s)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ ((~x) & z))

### Choose Function Test Cases

- - -

In [35]:
test_cases = [
    (0, 0, 0),  # x=0, choose z, 0
    (0, 0, 1),  # x=0, choose z, 1
    (0, 1, 0),  # x=0, choose z, 0
    (0, 1, 1),  # x=0, choose z, 1

    (1, 0, 0),  # x=1, choose y, 0
    (1, 0, 1),  # x=1, choose y, 0
    (1, 1, 0),  # x=1, choose y, 1
    (1, 1, 1),  # x=1, choose y, 1

    (0b0101, 0b1111, 0b0000),  # choose y at x=1 bits, 0b0101
    (0b1010, 0b1111, 0b0000),  # choose y at x=1 bits, 0b1010
]

for x, y, z in test_cases:
    result = Choose(x, y, z)
    print(f"x={x}, y={y}, z={z} = result={result:04b}")


x=0, y=0, z=0 = result=0000
x=0, y=0, z=1 = result=0001
x=0, y=1, z=0 = result=0000
x=0, y=1, z=1 = result=0001
x=1, y=0, z=0 = result=0000
x=1, y=0, z=1 = result=0000
x=1, y=1, z=0 = result=0001
x=1, y=1, z=1 = result=0001
x=5, y=15, z=0 = result=0101
x=10, y=15, z=0 = result=1010


## Majority Function Documentation

- - -

In [ ]:
def Majority(x,y,z):
    """
    Compute the bitwise majority function used in SHA-256.

    The majority function returns, for each bit position, the value that
    appears in at least two of the three 32-bit inputs. All inputs are
    converted to unsigned 32-bit integers to ensure correct wrapping and
    bitwise behaviour as defined in the Secure Hash Standard (FIPS 180-4).

    Parameters
    ----------
    x : int or np.uint32
        First input word.
    y : int or np.uint32
        Second input word.
    z : int or np.uint32
        Third input word.

    Returns
    -------
    np.uint32
        A 32-bit word where each bit is the majority bit of (x, y, z).

    Notes
    -----
    The function follows the SHA-256 definition:
    Maj(x, y, z) = (x & y) ^ (x & z) ^ (y & z)

    NumPy is used to enforce unsigned 32-bit arithmetic.

    Examples
    --------
    >>> Majority(0b1010, 0b1100, 0b0110)
    np.uint32(0b1010)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return np.uint32((x & y) ^ (x & z) ^ (y & z))

### Majority Function Test Cases

- - -

In [52]:
test_cases = [
    (0, 0, 0),  # all zeros → majority 0
    (0, 0, 1),  # two 0s, one 1 = majority 0
    (0, 1, 0),  # two 0s, one 1 = majority 0
    (0, 1, 1),  # two 1s, one 0 = majority 1
    (1, 0, 0),  # two 0s, one 1 = majority 0
    (1, 0, 1),  # two 1s, one 0 = majority 1
    (1, 1, 0),  # two 1s, one 0 = majority 1
    (1, 1, 1),  # all 1s = majority 1

    # multi-bit cases
    (0b0101, 0b1111, 0b0000),  # check bit-by-bit majority
    (0b1010, 0b1111, 0b0000),
]

for x, y, z in test_cases:
    result = Majority(x, y, z)
    print(f"x={x}, y={y}, z={z}  => {result:04b}")


x=0, y=0, z=0  => 0000
x=0, y=0, z=1  => 0000
x=0, y=1, z=0  => 0000
x=0, y=1, z=1  => 0001
x=1, y=0, z=0  => 0000
x=1, y=0, z=1  => 0001
x=1, y=1, z=0  => 0001
x=1, y=1, z=1  => 0001
x=5, y=15, z=0  => 0101
x=10, y=15, z=0  => 1010


### Sigma0 Function Implementation

---

In [ ]:
def Sigma0(x):
    """
    Compute the SHA-256 Big Sigma 0 function.

    This function rotates the 32-bit input word three times (by 2, 13, and 22
    bits) and XORs the results together. The input is always converted to an
    unsigned 32-bit integer so the rotations wrap around correctly, following
    the SHA-256 rules in FIPS 180-4.

    Parameters
    ----------
    x : int or np.uint32
        The input 32-bit word.

    Returns
    -------
    np.uint32
        The result of ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x).

    Notes
    -----
    Big Sigma 0 is one of the main mixing functions in SHA-256. NumPy is used
    to keep all operations within 32 bits.

    Examples
    --------
    >>> Sigma0(0b1010)
    np.uint32(…)
    """
    x = np.uint32(x)
    return np.uint32(
        ((x >> 2) | (x << (32 - 2))) ^
        ((x >> 13) | (x << (32 - 13))) ^
        ((x >> 22) | (x << (32 - 22)))
    )


### Sigma0 Test Cases

- - -

In [ ]:
test_cases = [
    0x00000000,
    0xFFFFFFFF,
    0x6A09E667,
    0x12345678,
    0x80000000,
    0x00000001,
]

for x in test_cases:
    result = Sigma0(x)
    print(f"x={x:08X}  ({x:032b})  =>  Σ0(x)={result:08X}  ({result:032b})")


x=00000000  (00000000000000000000000000000000)  =>  Σ0(x)=00000000  (00000000000000000000000000000000)
x=FFFFFFFF  (11111111111111111111111111111111)  =>  Σ0(x)=FFFFFFFF  (11111111111111111111111111111111)
x=6A09E667  (01101010000010011110011001100111)  =>  Σ0(x)=CE20B47E  (11001110001000001011010001111110)
x=12345678  (00010010001101000101011001111000)  =>  Σ0(x)=66146474  (01100110000101000110010001110100)
x=80000000  (10000000000000000000000000000000)  =>  Σ0(x)=20040200  (00100000000001000000001000000000)
x=00000001  (00000000000000000000000000000001)  =>  Σ0(x)=40080400  (01000000000010000000010000000000)


## Sigma1 Function Documentaion

---

In [ ]:
def Sigma1(x):
    """
    Compute the SHA-256 Big Sigma 1 function.

    This function rotates the 32-bit input word three times (by 6, 11, and 25
    bits) and XORs the results together. The input is converted to an unsigned
    32-bit integer so the rotations wrap around correctly, following the rules
    of the SHA-256 standard.

    Parameters
    ----------
    x : int or np.uint32
        The input 32-bit word.

    Returns
    -------
    np.uint32
        The result of ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x).

    Notes
    -----
    Big Sigma 1 is one of the main mixing functions in SHA-256. NumPy is used
    to keep the operations within 32 bits.

    Examples
    --------
    >>> Sigma1(0b1010)
    np.uint32(…)
    """
    
    x = np.uint32(x)

    return np.uint32(
    ((x >> 6) | (x << (32 - 6))) ^
    ((x >> 11) | (x << (32 - 11))) ^
    ((x >> 25) | (x << (32 - 25)))
    )

### Sigma1 Test Cases

- - -

In [ ]:
test_cases = [
    0x00000000,
    0xFFFFFFFF,
    0x6A09E667,
    0x12345678,
    0x80000000,
    0x00000001,
]

for x in test_cases:
    result = Sigma1(x)
    print(f"x={x:08X}  ({x:032b})  =>  Σ1(x)={result:08X}  ({result:032b})")


x=00000000  (00000000000000000000000000000000)  =>  Σ1(x)=00000000  (00000000000000000000000000000000)
x=FFFFFFFF  (11111111111111111111111111111111)  =>  Σ1(x)=FFFFFFFF  (11111111111111111111111111111111)
x=6A09E667  (01101010000010011110011001100111)  =>  Σ1(x)=55B65510  (01010101101101100101010100010000)
x=12345678  (00010010001101000101011001111000)  =>  Σ1(x)=3561ABDA  (00110101011000011010101111011010)
x=80000000  (10000000000000000000000000000000)  =>  Σ1(x)=02100040  (00000010000100000000000001000000)
x=00000001  (00000000000000000000000000000001)  =>  Σ1(x)=04200080  (00000100001000000000000010000000)


## sigma0 Function Implementation

---

In [ ]:
def sigma0(x):
    """
    Compute the SHA-256 small sigma 0 function.

    This function applies two right-rotations (by 7 and 18 bits) and one
    right-shift (by 3 bits) to the 32-bit input word, then XORs the results
    together. The input is converted to an unsigned 32-bit integer so all
    operations follow the 32-bit behaviour required by the SHA-256 standard.

    Parameters
    ----------
    x : int or np.uint32
        The input 32-bit word.

    Returns
    -------
    np.uint32
        The result of ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x).

    Notes
    -----
    Small sigma 0 is used in the message schedule part of SHA-256.
    NumPy ensures all shifts and rotations stay within 32 bits.

    Examples
    --------
    >>> sigma0(0b1010)
    np.uint32(…)
    """
    x = np.uint32(x)

    return np.uint32(
        ((x >> 7) | x << ((32 - 7))) ^
        ((x >> 18) | x << ((32 - 18))) ^
        ((x >> 3))
    )

### sigma0 Test Cases

- - - 

In [ ]:
test_cases = [
    0x00000000,
    0xFFFFFFFF,
    0x6A09E667,
    0x12345678,
    0x80000000,
    0x00000001,
]

for x in test_cases:
    result = sigma0(x)
    print(f"x={x:08X}  ({x:032b})  =>  σ0(x)={result:08X}  ({result:032b})")


x=00000000  (00000000000000000000000000000000)  =>  σ0(x)=00000000  (00000000000000000000000000000000)
x=FFFFFFFF  (11111111111111111111111111111111)  =>  σ0(x)=1FFFFFFF  (00011111111111111111111111111111)
x=6A09E667  (01101010000010011110011001100111)  =>  σ0(x)=BA0CF582  (10111010000011001111010110000010)
x=12345678  (00010010001101000101011001111000)  =>  σ0(x)=E7FCE6EE  (11100111111111001110011011101110)
x=80000000  (10000000000000000000000000000000)  =>  σ0(x)=11002000  (00010001000000000010000000000000)
x=00000001  (00000000000000000000000000000001)  =>  σ0(x)=02004000  (00000010000000000100000000000000)


## sigma1 Function Implementation

---

In [ ]:
def sigma1(x):
    """
    Compute the SHA-256 small sigma 1 function.

    This function applies two right-rotations (by 17 and 19 bits) and one
    right-shift (by 10 bits) to the 32-bit input word, then XORs the results
    together. The input is converted to an unsigned 32-bit integer so the
    operations follow the correct 32-bit behaviour required by SHA-256.

    Parameters
    ----------
    x : int or np.uint32
        The input 32-bit word.

    Returns
    -------
    np.uint32
        The result of ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x).

    Notes
    -----
    Small sigma 1 is used in the message schedule step of SHA-256.
    NumPy ensures that all shifts and rotations stay within 32 bits.

    Examples
    --------
    >>> sigma1(0b1010)
    np.uint32(…)
    """
    x = np.uint32(x)
    return np.uint32(
        ((x >> 17) | (x << (32 - 17))) ^
        ((x >> 19) | (x << (32 - 19))) ^
        (x >> 10)
    )

### sigma1 Test Cases

- - - 

In [ ]:
test_cases = [
    0x00000000,
    0xFFFFFFFF,
    0x6A09E667,
    0x12345678,
    0x80000000,
    0x00000001,
]

for x in test_cases:
    result = sigma1(x)
    print(f"x={x:08X}  ({x:032b})  =>  σ1(x)={result:08X}  ({result:032b})")


x=00000000  (00000000000000000000000000000000)  =>  σ1(x)=00000000  (00000000000000000000000000000000)
x=FFFFFFFF  (11111111111111111111111111111111)  =>  σ1(x)=003FFFFF  (00000000001111111111111111111111)
x=6A09E667  (01101010000010011110011001100111)  =>  σ1(x)=CFE5DA3C  (11001111111001011101101000111100)
x=12345678  (00010010001101000101011001111000)  =>  σ1(x)=A1F78649  (10100001111101111000011001001001)
x=80000000  (10000000000000000000000000000000)  =>  σ1(x)=00205000  (00000000001000000101000000000000)
x=00000001  (00000000000000000000000000000001)  =>  σ1(x)=0000A000  (00000000000000001010000000000000)


## Discussion / Interpretation

The results from my tests show that all seven functions behave exactly like the **Secure Hash Standard** describes. Converting every value to ``np.uint32`` kept the operations strictly within **32 bits**, which is important because **SHA-256** depends on fixed-size words. The outputs for simple inputs *(like all zeros or all ones)* matched what I expected, and the more complex inputs showed clear bit-mixing, which is the whole purpose of the **Sigma** and **sigma** functions.

Overall, my approach was correct and gave outputs consistent with real **SHA-256** behaviour. For now I used numpy to make the **32-bit** handling easier, but later I’d like to try implementing these functions without numpy, using manual masks and rotations. That would make the code more complex, but it would also help me understand how cryptographic code controls bit boundaries when the language doesn’t do it automatically.

## References

**Note:** *References I used universally for every problem can be found in the ``README.md`` under 'Universal References'*

*The following are sources I used in assistance for problem 1:*

- [Bitwise Operators in Python](https://realpython.com/python-bitwise-operators/) - *To get a understand bitwise operations further.*

- [Bitwise Operation](https://en.wikipedia.org/wiki/Bitwise_operation) - *To obtain a deeper understanding of bitwise operations.*

- [1.2 The Majority Function - Martin Hell](https://www.youtube.com/watch?v=g9G3AdnDxws) - *For a better understanding of the Majority function.* 


# Problem 2 - Fractional Parts of Cube Roots
- - -
## Question To Solve: 

Use **numpy** to calculate the constants listed at the bottom of page 11 of the **Secure Hash Standard**, following the steps below. These are the first `32 bits` of the fractional parts of the cube roots of the first 64 prime numbers.

1. Write a function called ``primes(n)`` that generates the **first n prime numbers**.

1. Use the function to **calculate the cube root of the first** ``64 primes``.

1. For each **cube root, extract the first ``32 bits`` of the fractional part**.

1. Display the result in **hexadecimal**.

1. Test the results against what is in the **Secure Hash Standard**.

## My Understanding 

For this problem, I need to recreate the constants that **SHA-256** uses in its compression function. **These constants come from the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers**, which is what the **[Secure Hash Standard describes on page 11](https://csrc.nist.gov/pubs/fips/180-2/final)**.

To do this, I **first need a function that can generate prime numbers, because the standard specifically uses the first 64 primes**. Once I have the primes, I must **calculate their cube roots**, and then **extract only the fractional part** *(everything after the decimal)*. After that, I **scale the fractional part so that I can take the first 32 bits of it, convert that into an integer, and display it in hexadecimal**. These hex values should match the constants shown in the official document.

Overall, this problem is about understanding how **SHA-256** chooses its constants. Even though the final values look random, they are actually produced by a mathematical. My job is to follow that rule exactly using **Python** and **NumPy**, and then compare my results to the values listed in the **Secure Hash Standard** to make sure my implementation is correct.

## Theory / Background

In **SHA-256**, some special numbers called **constants** are used to **mix and scramble the data during hashing**. These numbers are **not random**, they are made in a specific way using **prime numbers**.

The **Secure Hash Standard** explains that the constants come from the **fractional parts of the cube roots of the first 64 prime numbers**. This means you *take each prime number, find its cube root, ignore the whole number part, and keep only the decimal part. That decimal is then turned into a 32-bit number*.

Using primes makes sure the constants are hard to predict but still always the same. This gives **SHA-256** both security and consistency, the same input will always make the same hash, but it’s nearly impossible to guess how.

These constants are very important because they help cause the avalanche effect, where even a tiny change in the input creates a completely different hash result.

The idea of creating **constants from mathematical formulas like cuberoots and square roots of prime numbers** came from researches at the **U.S National Security Agency (NSA)**. The NSA originally designed the **Secure Hash Standard** family.

## Approach

For this problem, I needed to recreate the constants used by **SHA-256** from the **cube roots of the first 64 prime numbers**. To do this, I broke the task into small, easy steps.

First, I wrote a function called ``primes(n)`` that finds the **first ``n`` prime numbers using simple division**. It **checks each number starting from 2 and adds it to a list if it’s only divisible by 1 and itself**. It only checks if ``n`` is divisible by other numbers up until ``n``'s square root. I did this because if a number has a factor larger than it's square root, the other factor will have been found below. This was enough for the 64 primes needed, so there was no need for a more advanced method.

Next, I used **NumPy** to **find the cube root of each prime**. I then took only the **fractional part (everything after the decimal point) by subtracting the whole number part**. To get the **first 32 bits of that fractional value, I multiplied it by 2³² and converted it to a 32-bit integer using ``np.uint32``**.

Finally, I printed the constants in **hexadecimal**, which matches the format shown in the **Secure Hash Standard**.

I did it this way because it’s clear, follows the same steps described in the standard, and makes it easy to check that my calculated constants match the official SHA-256 ones. Using NumPy also made the cube root and math operations simple and accurate.


# Primes Function Implemetation

- - -

In [3]:
def primes(n):
    """
    Generate the first n prime numbers.

    This function creates a list containing the first n prime numbers.
    It starts checking from 2 (the first prime) and tests each number
    for divisibility up to its square root using simple division.
    If a number has no divisors other than 1 and itself, it is added to
    the list of primes. The process continues until n prime numbers are found.

    Parameters
    ----------
    n : int
        The number of prime numbers to generate.

    Returns
    -------
    list of int
        A list containing the first n prime numbers.

    Notes
    -----
    - Uses a basic trial division method for simplicity.
    - NumPy is used only for the square root calculation (`np.sqrt`).
    - This method is efficient enough for small values such as n = 64,
      which is required for the SHA-256 constants.

    Examples
    --------
    >>> primes(5)
    [2, 3, 5, 7, 11]
    >>> primes(10)[-1]
    29
    """
    primes_list = []
    num = 2  

    while len(primes_list) < n:
        for i in range(2, int(np.sqrt(num)) + 1):
            if num % i == 0:
                break
        else:
            primes_list.append(num)
        num += 1

    return primes_list


# Generate the first 64 primes
prime_numbers = primes(64)

# Calculate cube roots using NumPy
cube_roots = np.cbrt(prime_numbers)

# Extract fractional parts (ignore the whole number)
fractional_parts = cube_roots - np.floor(cube_roots)

# Take the first 32 bits of the fractional part
# multiply by 2^32 and convert to integers
constants = np.uint32(fractional_parts * (2**32))

# Display results in hexadecimal
for i, value in enumerate(constants):
    print(f"{i+1:02d}. Prime: {prime_numbers[i]:3d}  = Constant: 0x{value:08X}")

01. Prime:   2  = Constant: 0x428A2F98
02. Prime:   3  = Constant: 0x71374491
03. Prime:   5  = Constant: 0xB5C0FBCF
04. Prime:   7  = Constant: 0xE9B5DBA5
05. Prime:  11  = Constant: 0x3956C25B
06. Prime:  13  = Constant: 0x59F111F1
07. Prime:  17  = Constant: 0x923F82A4
08. Prime:  19  = Constant: 0xAB1C5ED5
09. Prime:  23  = Constant: 0xD807AA98
10. Prime:  29  = Constant: 0x12835B01
11. Prime:  31  = Constant: 0x243185BE
12. Prime:  37  = Constant: 0x550C7DC3
13. Prime:  41  = Constant: 0x72BE5D74
14. Prime:  43  = Constant: 0x80DEB1FE
15. Prime:  47  = Constant: 0x9BDC06A7
16. Prime:  53  = Constant: 0xC19BF174
17. Prime:  59  = Constant: 0xE49B69C1
18. Prime:  61  = Constant: 0xEFBE4786
19. Prime:  67  = Constant: 0x0FC19DC6
20. Prime:  71  = Constant: 0x240CA1CC
21. Prime:  73  = Constant: 0x2DE92C6F
22. Prime:  79  = Constant: 0x4A7484AA
23. Prime:  83  = Constant: 0x5CB0A9DC
24. Prime:  89  = Constant: 0x76F988DA
25. Prime:  97  = Constant: 0x983E5152
26. Prime: 101  = Constan

## Primes Function Test Cases

- - - 

In [4]:
# Generate small number of primes

print("Test 1: Generate first 10 primes")
print(primes(10))
# Expected output: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

# Edge case: Generate only one prime
print("Test 2: Generate first 10 primes")
print(primes(10))
# Expected output: [2]

# Test Cube Roots of Primes (Using exact same cuberoot check as implementation)
print("\nTest 4: Cube roots of first 5 primes")
sample_primes = [2, 3, 5, 7, 11]
cube_roots = np.cbrt(sample_primes)
print(np.round(cube_roots, 6))
# Expected approx: [1.259921, 1.442250, 1.709976, 1.913, 2.22398]

Test 1: Generate first 10 primes
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


## Discussion / Interpretation

I found this problem really interesting because it showed how something that first looks random in **SHA-256** actually comes from a clear and logical process. I liked seeing how math, like cube roots and prime numbers, can be used to make something secure and reliable. It made the algorithm feel less like *“magic”* and more like something I could actually understand.

Using **NumPy** made things easier, especially for cube roots and handling big numbers, and it felt satisfying to see my results match the real constants from the **Secure Hash Standard**. It was also nice to see that these constants aren’t secret or random — anyone can calculate them, which makes the algorithm more trustworthy.

Currently, the way I have the **cuberoot** and **fractional** calculations set up, it is quite difficult to run test cases. Maybe, I should place those calculations in their own seperate functions for easier testing.

Overall, I enjoyed this problem because it combined programming, logic, and math in a clear way, and it helped me understand how small details like these constants play an important role in cryptography.

## References

**Note:** *References I used universally for every problem can be found in the ``README.md`` under 'Universal References'*

*The following are sources I used in assistance for problem 2:*

- [List of prime numbers](https://en.wikipedia.org/wiki/List_of_prime_numbers) - *Used for reaffirming prime function output.*

- [Cuberoot Root Calculator](https://www.calculatorsoup.com/calculators/algebra/cuberoots.php) - *Checking if function outputting correct primes.*

- [Cuberoot Wikepedia](https://en.wikipedia.org/wiki/Cube_root) - *Information on cuberoots*

- [How to represent fractional numbers in Binary](https://www.luisllamas.es/en/represent-fractional-numbers-in-binary/) - *Used for understanding how to represent fractional numbers in binary.*

- [Hexadeximal Wiki](https://en.wikipedia.org/wiki/Hexadecimal) - *Information on hexadecimal representation* 

# Problem 3: Padding
- - - 

## Question To Solve

Write a **generator function** ``block_parse(msg)`` that processes messages according to section **5.1.1** and **5.2.1** of the **Secure Hash Standard**. The function should accept a **bytes object** called ``msg``. At each iteration, it should **yield the next 512-bit block** of ``msg`` as a bytes object. Ensure that the final block (or final two blocks) include(s) the required padding of ``msg`` as specified in the standard. Test the generator with messages of different lengths to confirm proper padding and block output.

## My Understanding

In this problem, I need to make a **generator function** called ``block_parse(msg)`` that splits a message into **512-bit blocks and adds the correct SHA-256 padding** at the end. A **generator function** in **Python** is a special type of function that uses the ``yield`` keyword instead of ``return``. This means it **doesn’t give all results at once** — it **produces one value at a time**, which is useful here because the message is processed in separate blocks.

According to sections **5.1.1** and **5.2.1** of the **Secure Hash Standard (FIPS 180-4)**, a **message in SHA-256 must be padded before hashing**. **Padding means extra bits are added to make the message length fit exactly into 512-bit blocks**. The rule says you always **add one 1 bit after the message**, followed by **enough 0s to make the total length 64 bits short of a multiple of 512**. Then, you add the original message length (in bits) as a **64-bit value at the very end**.

So, this function will take a message, pad it correctly, and then **yield each 512-bit chunk one at a time**. This process prepares the data for the main hashing steps in **SHA-256**. It’s a good way to learn how the algorithm structures its input before the actual hash calculations begin.

## Theory / Background

In **SHA-256**, the message can’t be hashed directly in one go. Instead, it is **split into fixed-size 512-bit blocks**. Real messages almost never fit perfectly into these block sizes, so we need padding. Padding is a way of adding extra bits in a standard, predictable way so that:

- Every possible original message has **one unique padded form**.

- The **final padded message length is a multiple of 512 bits**.

*For SHA-256, the padding rule is always the same:*

1. Start with the original message (in bits).

2. Append a single **1** bit.

3. **Append 0 bits until the length is 64 bits short of a multiple of 512**.

4. Finally, **append the original message length (in bits) as a 64-bit number**.

This idea comes from older hash function designs (like **MD4/MD5** and **SHA-1**) and was standardised in the **Secure Hash Standard (FIPS 180) by NIST**, based on designs originally created by the **NSA**. The goal is to make the padding rule public, simple, and safe, so there is no hidden trick in how messages are prepared.

On the **Python** side, a **generator function** is a function that can produce a sequence of values one at a time, instead of building everything in memory at once. Generators were introduced into **Python** by **Guido van Rossum** and others via **PEP 255**, first appearing in **Python 2.2**. They use the **yield** keyword to *“pause”* and *“resume”* execution.

In this problem, using a generator fits nicely: ``block_parse(msg)`` can yield one padded **512-bit block at a time**, which matches how **SHA-256** itself processes data internally—block by block—in a streaming fashion, rather than needing the whole padded message as a single big object.

## Approach

For this problem, my goal was to write a **generator** that prepares a message exactly the way **SHA-256** expects before hashing. I broke the task into simple steps that follow the padding rules from the Secure Hash Standard.

First, I calculated the original length of the message in bits, because **SHA-256** must store that at the very end. Then I added the required ``1`` bit, which is represented as the **byte 0x80**. After that, I **added zero bytes until the message length was exactly 64 bits away from a multiple of 512 bits**, which is what the standard requires.

Once that condition was met, I added the **64-bit length field (big-endian)**, which completes the padding. At that point, the padded message is **guaranteed to be a clean multiple of 512 bits**, so I simply sliced it into **64-byte blocks** and used ``yield`` to produce one block at a time. Using ``yield`` keeps the function efficient and makes it easier to process long messages without storing every block at once.

Overall, I followed the **SHA-256** rules step by step and wrote the generator in the simplest way possible, focusing on clarity and correctness rather than complexity.

# Padding Function Implementation

- - -

In [1]:
def block_parse(msg):
    """
    Split a message into 512-bit blocks with correct SHA-256 padding.

    This function takes a bytes message and pads it according to the
    SHA-256 rules. It adds the required '1' bit, then enough zeros, and
    finally the original message length in 64 bits. After padding, it
    produces each 512-bit (64-byte) block one at a time using `yield`,
    which makes the function a generator.

    Parameters
    ----------
    msg : bytes
        The original message to be padded and split into blocks.

    Returns
    -------
    bytes
        Each call to `yield` returns the next 64-byte block of the padded
        message.

    Notes
    -----
    - Follows the padding rules from the Secure Hash Standard (FIPS 180-4).
    - The message length is stored as a 64-bit big-endian value.
    - Padding ensures the final message size is a multiple of 512 bits.

    Examples
    --------
    >>> list(block_parse(b"abc"))
    [b'...64 bytes...', ...]
    """
    
    original_len_bits = len(msg) * 8

    # Append the '1' bit → which is 0x80 in bytes
    padded = msg + b"\x80"

    # Append zeros until the length is 64 bits short of a 512-bit block
    while (len(padded) * 8) % 512 != 448:
        padded += b"\x00"

    # Append the original message length as a 64-bit big-endian integer
    padded += original_len_bits.to_bytes(8, "big")

    # Yield each 512-bit (64-byte) block
    for i in range(0, len(padded), 64):
        yield padded[i:i+64]

## Padding Function Test Cases

- - -

In [2]:
# Empty message check
print("Test 1: Empty message")
for block in block_parse(b""):
    print(block, len(block))
  
# Short message check  
print("\nTest 2: Short message 'abc'")
for block in block_parse(b"abc"):
    print(block, len(block))
    

# Message that fits exactly one block (55 bytes)
print("\nTest 3: 55 bytes long")
msg = b"A" * 55
blocks = list(block_parse(msg))
print("Number of blocks:", len(blocks))
for b in blocks:
    print(len(b))

Test 1: Empty message
b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' 64

Test 2: Short message 'abc'
b'abc\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18' 64

Test 3: 55 bytes long
Number of blocks: 1
64


## Discussion / Interpretation

This problem helped me understand how **SHA-256** handles padding and why it’s such an important part of the hashing process. Writing the ``block_parse`` generator made it clear how the message has to be broken into **512-bit blocks (64 bytes)** and why the padding rules must be followed exactly for the hash to be correct.

I found the logic simple once it was broken down, add the 1 bit, add zeros, and then add the original length, but it still felt a bit tricky to get the ordering right on the first try. Using a generator also made sense for this kind of task because it naturally processes data one block at a time, which is how **SHA-256** works internally. It was interesting to see how ``yield`` allows the function to *“pause”* and ``return`` each block instead of holding everything in memory.

If I were to improve this, I might try handling the bytes manually without relying on **Python’s** built-in helpers, just to understand the bit-level operations more deeply. I could also try testing with streaming input or larger messages to see how the generator behaves in more realistic hashing situations. But overall, this problem made the padding rules feel much more understandable and less abstract.

## References

- [Real Python - Generators](https://realpython.com/introduction-to-python-generators/) - *Used to better understand how generators work and their applications.*

- [Real Python - Byte Objects](https://realpython.com/python-bytes/) - *Understand byte object better.*

- [MojoAuth - MD5 vs MD4](https://mojoauth.com/compare-hashing-algorithms/) - *Researching MD5 & MD4 cryptographic functions.* 

- [Wikipedia - Guido van Rossum](https://en.wikipedia.org/wiki/Guido_van_Rossum) - *Researching Python history for discussion section.*

- [Python Enhancement Proposals](https://peps.python.org/pep-0225/) - *Used for understanding the PEP 225 proposals for the discussion section.*

- [Wikipedia - Endianness](https://en.wikipedia.org/wiki/Endianness) - *For understanding endianness.*

- [W3Schools - Python Built in Functions](https://www.w3schools.com/python/python_ref_functions.asp) - *For finding functions I will need for this problem and how they work.*

# Problem 4: Hashes

## Question To Solve: 

Write a function ``hash(current, block)`` that calculates the next hash value given the ``current`` hash value and the next message ``block`` according to section **6.2.2 SHA-256 Hash Computation** on page 22 of the **Secure Hash Standard**.

## My Understanding

For this problem, I need to write a function called ``hash(current, block)`` that **follows the steps in Section 6.2.2 of the Secure Hash Standard**. My understanding is that hashing in **SHA-256** means taking data and repeatedly mixing it with a set of rules so the output becomes completely different from the input.

The word ``current`` represents the **temporary hash value at that stage of the algorithm**, and ``block`` is the **next 512-bit chunk of the message that needs to be processed**. The **SHA-256 algorithm updates the hash by running the block through several rounds of bitwise operations, rotations, additions**, and the functions defined earlier (like **Ch, Maj**, and the **Sigma** functions).

**Section 6.2.2** describes exactly how each ``block`` is combined with the current hash state. Each time we process a ``block``, we get a new **“current”** value, and this repeats until the whole message is done. **The final value becomes the final 256-bit hash**.

## Theory / Background


